In [19]:
import os, pandas as pd,numpy as np
path_future = "C:\\PICKLE\\Nifty Future"
path_options = "C:\\PICKLE\\Nifty Options"
# import warnings
# warnings.filterwarnings('ignore')

# List all files in the Nifty Future folder
future_files = [os.path.join(path_future, f) for f in os.listdir(path_future) if os.path.isfile(os.path.join(path_future, f))]
future_files = [f for f in future_files if "2024" in f]
# List all files in the Nifty Options folder
option_files = [os.path.join(path_options, f) for f in os.listdir(path_options) if os.path.isfile(os.path.join(path_options, f))]
option_files = [f for f in option_files if "2024" in f]

In [ ]:
def calculate_sl_intra(entry_time,sl,intra_sl,ce_data,pe_data):
    ce_pe_price = ce_data.loc[entry_time,'close']+pe_data.loc[entry_time,'close']

    sl_price = ce_pe_price*(1+sl/100)
    intra_sl_price = ce_pe_price*(1+intra_sl/100)
    return sl_price,ce_pe_price ,intra_sl_price

def check_sl_hit(ce_data,pe_data,sl_price,intra_sl_price):
    try:
        sl_time = ce_data[ce_data.close +  pe_data.close >= sl_price].index[0]
    except:
        sl_time = None
    try:
        intra_sl_time = ce_data[np.maximum(ce_data.high + pe_data.low, ce_data.low + pe_data.high )>= intra_sl_price].index[0]
    except:
        intra_sl_time = None

    return sl_time,intra_sl_time

def get_one_om(future_price,fpt,step):
    future_price = fpt['close'].iloc[0] if future_price is None else future_price

    return ((int(future_price/step)*step)/100)

def get_strangle_strike(start_dt, end_dt, opt: pd.DataFrame, fpt: pd.DataFrame, step: int, om=0.1, target=False, tf=1):
   
    valid_times = fpt.loc[start_dt:end_dt].index

    for current_dt in valid_times:
        future_price = fpt.loc[current_dt, 'close']
        one_om = get_one_om(future_price, fpt, step)
        target = one_om * om if not target else target
        
        # Filter options for the current time and whose closing price is >= the target
        target_od = opt[(opt.index == current_dt) & (opt['close'] >= target * tf)].sort_values(by=['close']).copy()

        # --- IMPORTANT: Handle the case where no options meet the criteria ---
        if target_od.empty:
            print(f"No options found for time {current_dt} that meet the target price criteria. Skipping.")
            continue  # Move to the next valid time

        try:
            ce_scrip = target_od.loc[target_od['scrip'].str.endswith('CE'), 'scrip'].iloc[0]
            pe_scrip = target_od.loc[target_od['scrip'].str.endswith('PE'), 'scrip'].iloc[0]
        except IndexError:
            # Handle cases where there is a CE but no PE (or vice versa)
            print(f"Could not find a matching CE/PE pair for time {current_dt}. Skipping.")
            continue

        ce_scrip_list = [ce_scrip, f"{int(ce_scrip[:-2]) - step}CE", f"{int(ce_scrip[:-2]) + step}CE"]
        pe_scrip_list = [pe_scrip, f"{int(pe_scrip[:-2]) - step}PE", f"{int(pe_scrip[:-2]) + step}PE"]

        call_list_prices, put_list_prices = [], []
        for z in range(3):
            try:
                # Get the close price for the call option
                call_list_prices.append(opt[opt['scrip'] == ce_scrip_list[z]].loc[current_dt, 'close'])
            except KeyError:
                call_list_prices.append(0)
            try:
                # Get the close price for the put option
                put_list_prices.append(opt[opt['scrip'] == pe_scrip_list[z]].loc[current_dt, 'close'])
            except KeyError:
                put_list_prices.append(0)
        
        # Initialize required_call and required_put with the first values
        required_call, required_put = call_list_prices[0], put_list_prices[0]
        min_diff = abs(required_put - required_call)
        
        target_2, target_3 = target * 2 * tf, target * 3

        # Check the initial pair
        if not ((required_put + required_call >= target_2) and (required_put + required_call <= target_3)):
            # Reset if initial pair does not meet criteria to find a better one
            required_call, required_put = None, None
            min_diff = float('inf')


        # Find the best combination of call and put from the lists
        for i in range(3):
            for j in range(3):
                current_call = call_list_prices[i]
                current_put = put_list_prices[j]
                
                # Ensure neither price is 0 (i.e., exists)
                if current_call > 0 and current_put > 0:
                    current_sum = current_call + current_put
                    current_diff = abs(current_put - current_call)
                    
                    if (current_sum >= target_2) and (current_sum <= target_3):
                        if current_diff < min_diff:
                            min_diff = current_diff
                            required_call = current_call
                            required_put = current_put
        
        # Check if a valid pair was found
        if required_call is not None and required_put is not None:
            # Get the scrip names for the found prices
            ce_scrip_final = ce_scrip_list[call_list_prices.index(required_call)]
            pe_scrip_final = pe_scrip_list[put_list_prices.index(required_put)]
            
            return ce_scrip_final, pe_scrip_final, required_call, required_put
        else:
            print(f"No suitable strangle pair found for time {current_dt} that meets the criteria. Skipping.")
            continue # Try the next current_dt
            
    # If the loop completes without finding a suitable pair, return None values
    return None, None, None, None

def backtest_strategy(opt, fut, om, entry_time, exit_time, sl, intra_sl):
    
    ce_scrip, pe_scrip, ce_price, pe_price = get_strangle_strike(entry_time, exit_time, opt, fut, 50, om=om, target=None, tf=1)
    
    opt = opt.loc[entry_time:exit_time]
    ce_data = opt[(opt['scrip'] == ce_scrip)]
    pe_data = opt[(opt['scrip'] == pe_scrip)]

    common_index = ce_data.index.intersection(pe_data.index)
    ce_data = ce_data.loc[common_index]
    pe_data = pe_data.loc[common_index]
    
    
    sl_price, ce_pe_price, intra_sl_price = calculate_sl_intra(entry_time, sl, intra_sl, ce_data, pe_data)
    print(f"sl price {sl_price} ce pe price {ce_pe_price}")
    
    total_pnl = 0
    trade_log = []
    current_entry_time = entry_time
    
    sl_time, intra_sl_time = check_sl_hit(ce_data, pe_data, sl_price, intra_sl_price)
    
    while True:
        trade_details = {
            'entry_time': current_entry_time,
            'ce_scrip': ce_scrip,
            'pe_scrip': pe_scrip,
            'entry_price': ce_pe_price
        }

        if sl_time is None and intra_sl_time is None:
            # Trade closes at the final exit time without hitting SL
            try:
                print("SL NOT HIT")
                exit_ce_pe = ce_data.loc[exit_time, 'close'] + pe_data.loc[exit_time, 'close']
                pnl = ce_pe_price - exit_ce_pe
                total_pnl += pnl
                trade_details.update({
                    'exit_time': exit_time,
                    'exit_price': exit_ce_pe,
                    'pnl': pnl,
                    'exit_reason': 'Time Exit'
                })
                trade_log.append(trade_details)
                print("total pnl ", total_pnl)
                break
            except Exception as e:
                print(e)
                print("SL NOT HIT")
                exit_ce_pe = ce_data.loc[ce_data.index[-1], 'close'] + pe_data.loc[pe_data.index[-1], 'close']
                pnl = ce_pe_price - exit_ce_pe
                total_pnl += pnl
                trade_details.update({
                    'exit_time': ce_data.index[-1],
                    'exit_price': exit_ce_pe,
                    'pnl': pnl,
                    'exit_reason': 'Time Exit (Partial Data)'
                })
                trade_log.append(trade_details)
                print("total pnl ", total_pnl)
                break

        elif sl_time is not None and intra_sl_time is not None:
            if sl_time < intra_sl_time:
                # Regular SL is hit
                print("SL HIT : ", sl_time)
                pnl = ce_pe_price - sl_price
                total_pnl += pnl
                trade_details.update({
                    'exit_time': sl_time,
                    'exit_price': sl_price,
                    'pnl': pnl,
                    'exit_reason': 'SL Hit'
                })
                trade_log.append(trade_details)
                print("pnl ", pnl)
                print('total pnl ', total_pnl)
                
                # Re-enter the next trade
                current_entry_time = sl_time
                sl_price, ce_pe_price, intra_sl_price = calculate_sl_intra(current_entry_time, sl, intra_sl, ce_data, pe_data)
                sl_time, intra_sl_time = check_sl_hit(ce_data, pe_data, sl_price, intra_sl_price)
                print("next trade hit")
            else:
                # Intra SL is hit
                print("INTRA SL HIT ", intra_sl)
                pnl = ce_pe_price - intra_sl_price
                trade_details.update({
                    'exit_time': intra_sl_time,
                    'exit_price': intra_sl_price,
                    'pnl': pnl,
                    'exit_reason': 'Intra SL Hit'
                })
                trade_log.append(trade_details)
                print("pnl ", pnl)
                break

        elif sl_time is not None and intra_sl_time is None:
            # Only regular SL is hit
            print("SL HIT : ", sl_time)
            pnl = ce_pe_price - sl_price
            total_pnl += pnl
            trade_details.update({
                'exit_time': sl_time,
                'exit_price': sl_price,
                'pnl': pnl,
                'exit_reason': 'SL Hit'
            })
            trade_log.append(trade_details)
            print("pnl ", pnl)
            print('total pnl ', total_pnl)
            
            # Re-enter the next trade
            current_entry_time = sl_time
            sl_price, ce_pe_price, intra_sl_price = calculate_sl_intra(current_entry_time, sl, intra_sl, ce_data, pe_data)
            sl_time, intra_sl_time = check_sl_hit(ce_data, pe_data, sl_price, intra_sl_price)
            print("next trade hit")

    # Create a DataFrame from the list of dictionaries
    trade_df = pd.DataFrame(trade_log)

    return {
        "total_pnl": total_pnl,
        "trade_details": trade_df
    }

In [14]:
nft = pd.read_pickle(future_files[0]).set_index('date_time')
opt = pd.read_pickle(option_files[0]).set_index('date_time')

date = str(nft.index[0]).split(" ")[0]
entry_time =  pd.to_datetime(f"{date} 09:20:00")
exit_time = pd.to_datetime(f"{date} 15:25:00")

sl ,intra_sl ,om=  8 , 20 , 0.1
print(om,entry_time,exit_time,sl,intra_sl)
backtest_strategy(opt,nft,om,entry_time,exit_time,sl,intra_sl)

0.1 2024-01-01 09:20:00 2024-01-01 15:25:00 8 20
sl price 59.832 ce pe price 55.4
SL HIT :  2024-01-01 14:11:00
pnl  -4.432000000000002
total pnl  -4.432000000000002
next trade hit
SL HIT :  2024-01-01 14:42:00
pnl  -4.936000000000007
total pnl  -9.36800000000001
next trade hit
SL HIT :  2024-01-01 14:44:00
pnl  -5.347999999999999
total pnl  -14.716000000000008
next trade hit
SL HIT :  2024-01-01 14:49:00
pnl  -5.847999999999999
total pnl  -20.564000000000007
next trade hit
SL NOT HIT
total pnl  24.73599999999999


{'total_pnl': np.float64(24.73599999999999),
 'trade_details':            entry_time ce_scrip pe_scrip  entry_price           exit_time  \
 0 2024-01-01 09:20:00  22000CE  21400PE        55.40 2024-01-01 14:11:00   
 1 2024-01-01 14:11:00  22000CE  21400PE        61.70 2024-01-01 14:42:00   
 2 2024-01-01 14:42:00  22000CE  21400PE        66.85 2024-01-01 14:44:00   
 3 2024-01-01 14:44:00  22000CE  21400PE        73.10 2024-01-01 14:49:00   
 4 2024-01-01 14:49:00  22000CE  21400PE        82.00 2024-01-01 15:25:00   
 
    exit_price     pnl exit_reason  
 0      59.832  -4.432      SL Hit  
 1      66.636  -4.936      SL Hit  
 2      72.198  -5.348      SL Hit  
 3      78.948  -5.848      SL Hit  
 4      36.700  45.300   Time Exit  }

In [ ]:
all_trades_df = pd.DataFrame()

trade_counter = 0

for nf, no in zip(future_files, option_files):
    try:
        nft = pd.read_pickle(nf).set_index('date_time')
        opt = pd.read_pickle(no).set_index('date_time')

        nft.index = pd.to_datetime(nft.index)
        opt.index = pd.to_datetime(opt.index)

        print(f"Processing file: {nf}")
        
        date = str(nft.index[0]).split(" ")[0]
        entry_time = pd.to_datetime(f"{date} 09:20:00")
        exit_time = pd.to_datetime(f"{date} 15:25:00")
        sl, intra_sl, om = 8, 20, 0.1

        # Call the backtest function and get the results
        result = backtest_strategy(opt, nft, om, entry_time, exit_time, sl, intra_sl)
        
        # Get the DataFrame of trade details from the result
        trade_details_df = result['trade_details']

        # Add a unique identifier for each day/file
        trade_details_df['file_id'] = trade_counter
        trade_counter += 1

        # Concatenate the new trades to the main DataFrame
        all_trades_df = pd.concat([all_trades_df, trade_details_df], ignore_index=True)

    except Exception as e:
        print(f"Error processing file {nf}: {e}")
        # You can log the error or just continue to the next file
        continue

# After the loop, save the master DataFrame to a single CSV file
if not all_trades_df.empty:
    all_trades_df['cumulative_pnl'] = all_trades_df['pnl'].cumsum()
    all_trades_df.to_csv('nifty/all_backtest_trades.csv', index=False)
    print("All backtest results saved to all_backtest_trades.csv")
else:
    print("No trade data was generated.")

Processing file: C:\PICKLE\Nifty Future\2024-01-01_nifty_future.pkl
sl price 59.832 ce pe price 55.4
SL HIT :  2024-01-01 14:11:00
pnl  -4.432000000000002
total pnl  -4.432000000000002
next trade hit
SL HIT :  2024-01-01 14:42:00
pnl  -4.936000000000007
total pnl  -9.36800000000001
next trade hit
SL HIT :  2024-01-01 14:44:00
pnl  -5.347999999999999
total pnl  -14.716000000000008
next trade hit
SL HIT :  2024-01-01 14:49:00
pnl  -5.847999999999999
total pnl  -20.564000000000007
next trade hit
SL NOT HIT
total pnl  24.73599999999999
Processing file: C:\PICKLE\Nifty Future\2024-01-02_nifty_future.pkl
sl price 49.03200000000001 ce pe price 45.400000000000006
SL HIT :  2024-01-02 09:58:00
pnl  -3.632000000000005
total pnl  -3.632000000000005
next trade hit
SL HIT :  2024-01-02 10:24:00
pnl  -4.068000000000005
total pnl  -7.70000000000001
next trade hit
INTRA SL HIT  20
pnl  -11.54
Processing file: C:\PICKLE\Nifty Future\2024-01-03_nifty_future.pkl
sl price 61.937999999999995 ce pe price 57